Author: Matthew Kinsley

Class: CIS663

Date: 2021-02-25


# Overview

This file will build on the code and teaching by Valerio Velardo in which he provide a bare bones and simple example code for MFCC feature extraction using librosa.

His github post can be found at.
https://github.com/musikalkemist/Deep-Learning-Audio-Application-From-Design-to-Deployment/tree/master/3-%20Implementing%20a%20Speech%20Recognition%20System%20in%20TensorFlow%202

We have adapted his code to work with our dataset.

# Global Imports
The following contains the global imports for the notebook.

In [1]:
import os
import json
import wave
import shutil
import librosa
import collections
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn.model_selection import train_test_split

# Global Constants
The following code contains the constants that are used throughout the file.

In [2]:
DATASET_PATH = "../../data/wav48"
WORK_PATH = '../../data/working'    # Working folder to use for blocks.
JSON_PATH = "."
EXP_SUBJECTS = 10                   # The number of subjects to process so we don't 
                                    # test on all which would take a long time.

# Shared Methods
Teh follwing code block defines shared methods that can be used throughout this notebook.

In [3]:
# ------------------------------------------------------------------------------------------------
# Description:
#    This function finds the next file in the folder that hasn't been processed
# yet.
#
# Inputs:
#    data_path - The path where files are stored
#    idx - The index of the suspected next file
#
# Output:
#    is_good - True if a file was found, otherwise False
#    file_path - The full file name and path string.
#    idx - The suspected idx of the next file
# ------------------------------------------------------------------------------------------------
def get_next_file_name(data_path, idx):
    is_good = False
    file_path = data_path + '_' + str(idx).zfill(3) + '.wav'

    # Give it 3 attempts to find a good file because some file ids are missing
    x = 0
    while not Path(file_path).exists() and x < 3:
        idx = idx+1
        file_path = data_path + '_' + str(idx).zfill(3) + '.wav'
        x = x+1

    # If we found a file make sure is_good is true
    if Path(file_path).exists():
        is_good = True

    return is_good, file_path, idx+1

In [4]:
# ------------------------------------------------------------------------------------------------
# Description:
#    This function loads the original waveforms and then blocks them into the  
# specified number of blocks eccach of a specified length.
#
# Paramaters:
#    data_path - The path where files are stored
#    working_path - The path were the working files are created.
#    blocks - The number of files to create for each subject.
#    len - The number of seconds each files should be.
#    max_subj - Specifies the maximum number of subjects to process
#
# Outputs:
#    DataFrame containing the subject ID and file name for each file created.
# ------------------------------------------------------------------------------------------------
def prep_wave_files(data_path, working_path, len_sec, max_subj=-1):
    # Define named tuple
    wparams = collections.namedtuple('WParams', 'nchannels sampwidth, framerate, nframes, comptype, compname')

    # Delete and files in the working path that may be left over
    try:
        shutil.rmtree(working_path);
    except:
        print('No working path')
    os.mkdir(working_path)
    
    # Get a listing of all dir entries in the datapath    
    entries = os.scandir(data_path)

    # Each directory represents a subject.  We need to load those files an build
    # the necessary amount of 60 second files.  
    subj = 0
    for entry in entries:
        if subj < max_subj or max_subj==-1:
            subj = subj+1
            subject_id = entry.name
            path = entry.path

            out_path = working_path + '/' + subject_id
            os.mkdir(out_path)

            print('    Preparing Subject ', subject_id)

            j = 1
            bcnt = 0
            audiof = []
            done = False
            while not done:
                out_file_name = out_path + '/' + subject_id + '_' + str(bcnt).zfill(3) + '.wav'
                is_good, in_file_name, j = get_next_file_name(path + '/' + subject_id, j)

                # If good then read in the wave file
                if is_good:
                    # read in the file
                    wf = wave.open(in_file_name, 'rb')

                    # Downsample it
                    p = wf.getparams()
                    data = wf.readframes(wf.getnframes())

                    # Append it to the working stream
                    audiof.append([p, data])
                    sr = p.framerate
                    wf.close()

                    # Check to see if we have exceeded 60 seconds yet
                    length = 0
                    for i in range(0,len(audiof)):
                        length = length + len(audiof[i][1])

                    if length > len_sec*sr*2:
                        # Calculate important parameters for writing the file
                        l_block = len(audiof)-1
                        l_block_overflow = (length-len_sec*sr*2)
                        l_block_len = len(audiof[l_block][1])-l_block_overflow

                        # Write the block
                        output = wave.open(out_file_name, 'wb')
                        output.setparams(audiof[0][0])
                        for i in range(0, len(audiof)-1):
                            output.writeframes(audiof[i][1])

                        output.writeframes(audiof[l_block][1][0:l_block_len])
                        output.close()

                        # Increment the block count
                        # Save the remainder
                        audiof = []
                        bcnt = bcnt + 1
                else:
                    done = True

            print('    Subject ', subject_id, ' Complete')

In [5]:
# ------------------------------------------------------------------------------------------------
# Description:
#    This method orginally produced by Valerio Velardo uses librosa to extract MFCC from all the 
# audo files in a given folder and specified parameters.  All extracted features are exprted to a
# json file.  It has been modified to work with our dataset.
#
# Note: This method requires that all audio files for each individual are stored in the same folder
# It uses the folder name to determine that a file belongs to an individual.  This aligns with the
# structure of the dataset and can be run without chagnes.
#
# Parameters:
#    dataset_path - The path to the root folder to search for audio files.
#    jason_path - Full path to the json file to fill with the extacted features.
#    num_mfcc - The number of coefficients to extract
#    n_fft - Interval to use to apply the FFT.  Measured in $ of samples.
#    hop_length - Slidding window for FFT measured in # of samples.
#    max_subj - Specifies the maximum number of subjects to process
# ------------------------------------------------------------------------------------------------
def preprocess_dataset(dataset_path, json_path, min_samples=22050, num_mfcc=13, n_fft=2048, hop_length=512,
                       max_subj=-1):
    # dictionary where we'll store mapping, labels, MFCCs and filenames
    data = {
        "mapping": [],
        "labels": [],
        "MFCCs": [],
        "files": []
    }

    # Get a listing of all dir entries in the datapath    
    entries = os.scandir(dataset_path)

    subj = 0
    for entry in entries:
        # make sure we don't process too many subjects
        if subj < max_subj or max_subj==-1:
            subj = subj+1
            subject_id = entry.name
            path = entry.path
            print("    Processing: '{}'".format(subject_id))

            j = 1
            bcnt = 0
            done = False
            while not done:
                is_good, in_file_name, j = get_next_file_name(path + '/' + subject_id, j)

                # If good then read in the wave file
                if is_good:
                    # save label (i.e., sub-folder name) in the mapping
                    data["mapping"].append(subject_id)

                    # Original code stopped processing when it hit a non audio file.  Adding the 
                    # exception handler allows the file to be logged and the processing to continue.
                    try:
                        # load audio file and slice it to ensure length consistency among different files
                        signal, sample_rate = librosa.load(in_file_name)

                        # drop audio files with less than pre-decided number of samples
                        if len(signal) >= min_samples:

                            # ensure consistency of the length of the signal
                            signal = signal[:min_samples]

                            # extract MFCCs
                            MFCCs = librosa.feature.mfcc(signal, sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
                                                        hop_length=hop_length)

                            # store data for analysed track
                            data["MFCCs"].append(MFCCs.T.tolist())
                            data["labels"].append(subj-1)
                            data["files"].append(in_file_name)
                    except:
                        print('    Can not load: {}: {}'.format(in_file_name, subj-1))
                else:
                    done = True
        
    # save data in json file
    print('    Saving json file')
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)

In [6]:
# ------------------------------------------------------------------------------------------------
# Description:
#    This method loads the input and target data from the specified json file.  
# Originally created by Valerio Velardo.
#
# Parameters:
#    data_path - Path to the json file containing the data
#
# Outputs: 
#    X - input data
#    y - target data
# ------------------------------------------------------------------------------------------------
def load_data(data_path):
    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["MFCCs"])
    y = np.array(data["labels"])
    print("Training sets loaded!")
    return X, y


In [7]:
# ------------------------------------------------------------------------------------------------
# Description:
#    This creates the training and validation splits that will be used for this generic training
# pass.  Adapted from a method created by Valerio Velardo.
#
# Parameters:
#    data_path - Path to the json file containing data
#    validation_size - Describes what percentage to use for the validation set size.
#
# Outputs: 
#    x_train - The training inputs
#    y_train  - The training targets
#    x_val - The validation inputs
#    y_val - The validation targets
# ------------------------------------------------------------------------------------------------
def prepare_dataset(data_path, validation_size=0.2):
    # load dataset
    X, y = load_data(data_path)

    # create train, validation, test split
    X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=validation_size)

    # add an axis to nd array
    X_train = X_train[..., np.newaxis]
    X_validation = X_validation[..., np.newaxis]

    return X_train, y_train, X_validation, y_validation

In [8]:
# ------------------------------------------------------------------------------------------------
# Description:
#    This method orginally produced by Valerio Velardo ubuilds a simple convolutional model in 
# keras that can be used to train for speaker identification from the dataset stored in a json 
# file.
#
# Parameters:
#    input_shape - Is a touple representing the shape of a training sample.
#    output_shape - Is the shape of the output layer
#    loss - A string representing which loss funciton to use
#    learning_rate - A fload specifying the learning rate.
#
# Outputs:
#    model - The tensorflow model
# ------------------------------------------------------------------------------------------------
def build_model(input_shape, output_shape, loss="sparse_categorical_crossentropy", learning_rate=0.0001):
    # build network architecture using convolutional layers
    model = tf.keras.models.Sequential()

    # 1st conv layer
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape,
                                     kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))

    # 2nd conv layer
    model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu',
                                     kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))

    # 3rd conv layer
    model.add(tf.keras.layers.Conv2D(32, (2, 2), activation='relu',
                                     kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2), padding='same'))

    # flatten output and feed into dense layer
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    tf.keras.layers.Dropout(0.3)

    # softmax output layer
    model.add(tf.keras.layers.Dense(output_shape, activation='softmax'))

    optimiser = tf.optimizers.Adam(learning_rate=learning_rate)

    # compile model
    model.compile(optimizer=optimiser,
                  loss=loss,
                  metrics=["accuracy"])

    return model

In [9]:
# ------------------------------------------------------------------------------------------------
# Description:
#    This method orginally produced by Valerio Velardo trains a model and returns the training
# history.
#
# Parameters:
#    model - The model to train
#    epochs - The number of epochs
#    batch_size - The size of each batch
#    patience - The number of epochs before early stop is allowed
#    x_train - The trianing inputs
#    y_train - The training targets
#    x_validation - The validation inputs
#    y_validation - The validation targets
#
# Outputs:
#    history the training history
# ------------------------------------------------------------------------------------------------
def train(model, epochs, batch_size, patience, X_train, y_train, X_validation, y_validation):
    # Setup the earlystop callback
    earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="accuracy", min_delta=0.001, patience=patience)

    # train model
    history = model.fit(X_train,
                        y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(X_validation, y_validation),
                        callbacks=[earlystop_callback])
    return history

# Extract Features
The following code extracts a series of inputs that can be used to build models.  To do this we will extract the files in a couple of different ways.  We will then mix them up all the way.  Once that is done we will test each of those inputs by running them through a simple model with minimumal training iterations to see which set of data creates the best generalized model.  

For this section the inputs will be as follows.
- Length of speach
    - Raw Files
    - 30 seconds
    - 60 seconds


- num_mfccs
    - 13
    - 20
    - 30


- n_fft
    - 1024
    - 2048
    - 4096


- hop_length
    - 256
    - 512
    - 1024
    
    
The first step is to extract all audo files into various jason files.

In [10]:
# List of constants to use to vary the extraction
audio_length = [0, 30, 60]
num_mfccs = [13, 20, 30]
n_fft = [1024, 2048, 4096]
hop_length = [256, 512, 1024]

# List of all create json files
json_files = []

# Extract the files to various json files
for al in audio_length:
    for nm in num_mfccs:
        for nf in n_fft:
            for hl in hop_length:
                json_file_name = 'data_{0}_{1}_{2}_{3}.json'.format(al, nm, nf, hl)
                json_files.append(json_file_name)
                
                print('building data for', json_file_name)
                dsPath = DATASET_PATH
                
                # Merge files to the specified length as needed
                if al != 0:
                    prep_wave_files(dsPath, WORK_PATH, al, max_subj=EXP_SUBJECTS)
                    dsPath = WORK_PATH
                    
                # Extract the features into the json file
                preprocess_dataset(dsPath, json_file_name, num_mfcc=nm, n_fft=nf, hop_length=hl, 
                                   max_subj=EXP_SUBJECTS)

building data for data_0_13_1024_256.json
    Processing: 'p226'
    Processing: 'p302'
    Processing: 'p284'
    Processing: 'p239'
    Processing: 'p335'
    Processing: 'p263'
    Processing: 'p283'
    Processing: 'p282'
    Processing: 'p276'
    Processing: 'p362'
    Saving json file
building data for data_0_13_1024_512.json
    Processing: 'p226'
    Processing: 'p302'
    Processing: 'p284'
    Processing: 'p239'
    Processing: 'p335'
    Processing: 'p263'
    Processing: 'p283'
    Processing: 'p282'
    Processing: 'p276'
    Processing: 'p362'
    Saving json file
building data for data_0_13_1024_1024.json
    Processing: 'p226'
    Processing: 'p302'
    Processing: 'p284'
    Processing: 'p239'
    Processing: 'p335'
    Processing: 'p263'
    Processing: 'p283'
    Processing: 'p282'
    Processing: 'p276'
    Processing: 'p362'
    Saving json file
building data for data_0_13_2048_256.json
    Processing: 'p226'
    Processing: 'p302'
    Processing: 'p284'
    Proce

    Preparing Subject  p263
    Subject  p263  Complete
    Preparing Subject  p283
    Subject  p283  Complete
    Preparing Subject  p282
    Subject  p282  Complete
    Preparing Subject  p276
    Subject  p276  Complete
    Preparing Subject  p362
    Subject  p362  Complete
    Processing: 'p226'
    Processing: 'p302'
    Processing: 'p284'
    Processing: 'p239'
    Processing: 'p335'
    Processing: 'p263'
    Processing: 'p283'
    Processing: 'p282'
    Processing: 'p276'
    Processing: 'p362'
    Saving json file
building data for data_30_13_1024_512.json
    Preparing Subject  p226
    Subject  p226  Complete
    Preparing Subject  p302
    Subject  p302  Complete
    Preparing Subject  p284
    Subject  p284  Complete
    Preparing Subject  p239
    Subject  p239  Complete
    Preparing Subject  p335
    Subject  p335  Complete
    Preparing Subject  p263
    Subject  p263  Complete
    Preparing Subject  p283
    Subject  p283  Complete
    Preparing Subject  p282
    Su

    Saving json file
building data for data_30_20_1024_512.json
    Preparing Subject  p226
    Subject  p226  Complete
    Preparing Subject  p302
    Subject  p302  Complete
    Preparing Subject  p284
    Subject  p284  Complete
    Preparing Subject  p239
    Subject  p239  Complete
    Preparing Subject  p335
    Subject  p335  Complete
    Preparing Subject  p263
    Subject  p263  Complete
    Preparing Subject  p283
    Subject  p283  Complete
    Preparing Subject  p282
    Subject  p282  Complete
    Preparing Subject  p276
    Subject  p276  Complete
    Preparing Subject  p362
    Subject  p362  Complete
    Processing: 'p226'
    Processing: 'p302'
    Processing: 'p284'
    Processing: 'p239'
    Processing: 'p335'
    Processing: 'p263'
    Processing: 'p283'
    Processing: 'p282'
    Processing: 'p276'
    Processing: 'p362'
    Saving json file
building data for data_30_20_1024_1024.json
    Preparing Subject  p226
    Subject  p226  Complete
    Preparing Subject  p3

    Subject  p276  Complete
    Preparing Subject  p362
    Subject  p362  Complete
    Processing: 'p226'
    Processing: 'p302'
    Processing: 'p284'
    Processing: 'p239'
    Processing: 'p335'
    Processing: 'p263'
    Processing: 'p283'
    Processing: 'p282'
    Processing: 'p276'
    Processing: 'p362'
    Saving json file
building data for data_30_30_1024_1024.json
    Preparing Subject  p226
    Subject  p226  Complete
    Preparing Subject  p302
    Subject  p302  Complete
    Preparing Subject  p284
    Subject  p284  Complete
    Preparing Subject  p239
    Subject  p239  Complete
    Preparing Subject  p335
    Subject  p335  Complete
    Preparing Subject  p263
    Subject  p263  Complete
    Preparing Subject  p283
    Subject  p283  Complete
    Preparing Subject  p282
    Subject  p282  Complete
    Preparing Subject  p276
    Subject  p276  Complete
    Preparing Subject  p362
    Subject  p362  Complete
    Processing: 'p226'
    Processing: 'p302'
    Processing:

    Subject  p284  Complete
    Preparing Subject  p239
    Subject  p239  Complete
    Preparing Subject  p335
    Subject  p335  Complete
    Preparing Subject  p263
    Subject  p263  Complete
    Preparing Subject  p283
    Subject  p283  Complete
    Preparing Subject  p282
    Subject  p282  Complete
    Preparing Subject  p276
    Subject  p276  Complete
    Preparing Subject  p362
    Subject  p362  Complete
    Processing: 'p226'
    Processing: 'p302'
    Processing: 'p284'
    Processing: 'p239'
    Processing: 'p335'
    Processing: 'p263'
    Processing: 'p283'
    Processing: 'p282'
    Processing: 'p276'
    Processing: 'p362'
    Saving json file
building data for data_60_13_2048_256.json
    Preparing Subject  p226
    Subject  p226  Complete
    Preparing Subject  p302
    Subject  p302  Complete
    Preparing Subject  p284
    Subject  p284  Complete
    Preparing Subject  p239
    Subject  p239  Complete
    Preparing Subject  p335
    Subject  p335  Complete
    Pr

    Processing: 'p335'
    Processing: 'p263'
    Processing: 'p283'
    Processing: 'p282'
    Processing: 'p276'
    Processing: 'p362'
    Saving json file
building data for data_60_20_2048_256.json
    Preparing Subject  p226
    Subject  p226  Complete
    Preparing Subject  p302
    Subject  p302  Complete
    Preparing Subject  p284
    Subject  p284  Complete
    Preparing Subject  p239
    Subject  p239  Complete
    Preparing Subject  p335
    Subject  p335  Complete
    Preparing Subject  p263
    Subject  p263  Complete
    Preparing Subject  p283
    Subject  p283  Complete
    Preparing Subject  p282
    Subject  p282  Complete
    Preparing Subject  p276
    Subject  p276  Complete
    Preparing Subject  p362
    Subject  p362  Complete
    Processing: 'p226'
    Processing: 'p302'
    Processing: 'p284'
    Processing: 'p239'
    Processing: 'p335'
    Processing: 'p263'
    Processing: 'p283'
    Processing: 'p282'
    Processing: 'p276'
    Processing: 'p362'
    Savi

    Subject  p283  Complete
    Preparing Subject  p282
    Subject  p282  Complete
    Preparing Subject  p276
    Subject  p276  Complete
    Preparing Subject  p362
    Subject  p362  Complete
    Processing: 'p226'
    Processing: 'p302'
    Processing: 'p284'
    Processing: 'p239'
    Processing: 'p335'
    Processing: 'p263'
    Processing: 'p283'
    Processing: 'p282'
    Processing: 'p276'
    Processing: 'p362'
    Saving json file
building data for data_60_30_2048_512.json
    Preparing Subject  p226
    Subject  p226  Complete
    Preparing Subject  p302
    Subject  p302  Complete
    Preparing Subject  p284
    Subject  p284  Complete
    Preparing Subject  p239
    Subject  p239  Complete
    Preparing Subject  p335
    Subject  p335  Complete
    Preparing Subject  p263
    Subject  p263  Complete
    Preparing Subject  p283
    Subject  p283  Complete
    Preparing Subject  p282
    Subject  p282  Complete
    Preparing Subject  p276
    Subject  p276  Complete
    Pr

Next we run each of these input files through the basic model with the same parameters and extract the performance metrics.

In [11]:
results = []

for fName in json_files:
    print('Processing', fName)
    
    # generate train, validation and test sets
    x_train, y_train, x_validation, y_validation = prepare_dataset(fName)

    # extract the shapes
    input_shape = (x_train.shape[1], x_train.shape[2], 1)
    output_count = np.unique(y_train).shape[0]
    
    print('input shape:', x_train.shape[1], x_train.shape[2])
    print('output shape:', output_count)
    
    # create network
    model = build_model(input_shape, output_count, learning_rate=0.0001)

    # train the network we will great each set of inputs based  on theor performance on a fixed number of epoch
    # and batch size to get a feel for which has the best descriptive power.
    history = train(model, 10, 32, 5, x_train, y_train, x_validation, y_validation)

    results.append({'filename':fName, 'history':history, 'model':model})

Processing data_0_13_1024_256.json
Training sets loaded!
input shape: 87 13
output shape: 10
Epoch 1/10
106/106 [==============================] - 6s 55ms/step - loss: 2.5716 - accuracy: 0.1529 - val_loss: 2.3199 - val_accuracy: 0.1568
Epoch 2/10
106/106 [==============================] - 5s 50ms/step - loss: 1.9786 - accuracy: 0.3278 - val_loss: 2.0213 - val_accuracy: 0.3124
Epoch 3/10
106/106 [==============================] - 5s 51ms/step - loss: 1.6788 - accuracy: 0.4623 - val_loss: 1.7334 - val_accuracy: 0.4299
Epoch 4/10
106/106 [==============================] - 5s 51ms/step - loss: 1.4718 - accuracy: 0.5428 - val_loss: 1.5121 - val_accuracy: 0.5154
Epoch 5/10
106/106 [==============================] - 5s 50ms/step - loss: 1.3025 - accuracy: 0.5947 - val_loss: 1.3334 - val_accuracy: 0.5760
Epoch 6/10
106/106 [==============================] - 6s 53ms/step - loss: 1.1744 - accuracy: 0.6366 - val_loss: 1.2103 - val_accuracy: 0.6223
Epoch 7/10
106/106 [=============================

106/106 [==============================] - 1s 13ms/step - loss: 1.8229 - accuracy: 0.4130 - val_loss: 1.8429 - val_accuracy: 0.4240
Epoch 5/10
106/106 [==============================] - 1s 12ms/step - loss: 1.7105 - accuracy: 0.4667 - val_loss: 1.7081 - val_accuracy: 0.4727
Epoch 6/10
106/106 [==============================] - 1s 12ms/step - loss: 1.6012 - accuracy: 0.5062 - val_loss: 1.6172 - val_accuracy: 0.5059
Epoch 7/10
106/106 [==============================] - 1s 13ms/step - loss: 1.5113 - accuracy: 0.5211 - val_loss: 1.5222 - val_accuracy: 0.5428
Epoch 8/10
106/106 [==============================] - 1s 12ms/step - loss: 1.4231 - accuracy: 0.5683 - val_loss: 1.4432 - val_accuracy: 0.5629
Epoch 9/10
106/106 [==============================] - 1s 13ms/step - loss: 1.3493 - accuracy: 0.5864 - val_loss: 1.3674 - val_accuracy: 0.5903
Epoch 10/10
106/106 [==============================] - 2s 15ms/step - loss: 1.2667 - accuracy: 0.6140 - val_loss: 1.2953 - val_accuracy: 0.6152
Processin

106/106 [==============================] - 4s 41ms/step - loss: 1.1239 - accuracy: 0.6553 - val_loss: 1.1723 - val_accuracy: 0.6401
Epoch 9/10
106/106 [==============================] - 4s 39ms/step - loss: 1.0434 - accuracy: 0.6820 - val_loss: 1.1212 - val_accuracy: 0.6591
Epoch 10/10
106/106 [==============================] - 4s 40ms/step - loss: 0.9686 - accuracy: 0.7090 - val_loss: 1.0964 - val_accuracy: 0.6651
Processing data_0_20_1024_1024.json
Training sets loaded!
input shape: 22 20
output shape: 10
Epoch 1/10
106/106 [==============================] - 2s 21ms/step - loss: 2.5393 - accuracy: 0.1455 - val_loss: 2.4770 - val_accuracy: 0.1449
Epoch 2/10
106/106 [==============================] - 2s 20ms/step - loss: 2.1221 - accuracy: 0.2571 - val_loss: 2.1435 - val_accuracy: 0.2221
Epoch 3/10
106/106 [==============================] - 2s 20ms/step - loss: 1.9107 - accuracy: 0.3545 - val_loss: 1.9346 - val_accuracy: 0.3420
Epoch 4/10
106/106 [==============================] - 2s 2

106/106 [==============================] - 4s 40ms/step - loss: 2.4607 - accuracy: 0.1841 - val_loss: 2.3205 - val_accuracy: 0.1366
Epoch 2/10
106/106 [==============================] - 4s 39ms/step - loss: 1.9176 - accuracy: 0.3584 - val_loss: 1.9652 - val_accuracy: 0.3409
Epoch 3/10
106/106 [==============================] - 4s 39ms/step - loss: 1.6631 - accuracy: 0.4786 - val_loss: 1.7045 - val_accuracy: 0.4632
Epoch 4/10
106/106 [==============================] - 4s 39ms/step - loss: 1.4339 - accuracy: 0.5638 - val_loss: 1.5035 - val_accuracy: 0.5511
Epoch 5/10
106/106 [==============================] - 4s 39ms/step - loss: 1.2454 - accuracy: 0.6265 - val_loss: 1.3163 - val_accuracy: 0.5974
Epoch 6/10
106/106 [==============================] - 4s 40ms/step - loss: 1.0891 - accuracy: 0.6811 - val_loss: 1.1718 - val_accuracy: 0.6437
Epoch 7/10
106/106 [==============================] - 4s 39ms/step - loss: 0.9611 - accuracy: 0.7245 - val_loss: 1.0681 - val_accuracy: 0.6722
Epoch 8/10

Epoch 5/10
106/106 [==============================] - 13s 122ms/step - loss: 0.7258 - accuracy: 0.7939 - val_loss: 0.8058 - val_accuracy: 0.7732
Epoch 6/10
106/106 [==============================] - 13s 122ms/step - loss: 0.5971 - accuracy: 0.8337 - val_loss: 0.6919 - val_accuracy: 0.7922
Epoch 7/10
106/106 [==============================] - 13s 121ms/step - loss: 0.5097 - accuracy: 0.8643 - val_loss: 0.6282 - val_accuracy: 0.8290
Epoch 8/10
106/106 [==============================] - 13s 126ms/step - loss: 0.4474 - accuracy: 0.8830 - val_loss: 0.5871 - val_accuracy: 0.8219
Epoch 9/10
106/106 [==============================] - 13s 123ms/step - loss: 0.3995 - accuracy: 0.8970 - val_loss: 0.5351 - val_accuracy: 0.8468
Epoch 10/10
106/106 [==============================] - 13s 122ms/step - loss: 0.3473 - accuracy: 0.9189 - val_loss: 0.5009 - val_accuracy: 0.8610
Processing data_0_30_2048_512.json
Training sets loaded!
input shape: 44 30
output shape: 10
Epoch 1/10
106/106 [================

...

6/6 [==============================] - 0s 83ms/step - loss: 3.4503 - accuracy: 0.0615 - val_loss: 5.4599 - val_accuracy: 0.0889
Epoch 2/10
6/6 [==============================] - 0s 50ms/step - loss: 3.1183 - accuracy: 0.1006 - val_loss: 4.3732 - val_accuracy: 0.1333
Epoch 3/10
6/6 [==============================] - 0s 47ms/step - loss: 2.9169 - accuracy: 0.1341 - val_loss: 3.8828 - val_accuracy: 0.1333
Epoch 4/10
6/6 [==============================] - 0s 46ms/step - loss: 2.7753 - accuracy: 0.1341 - val_loss: 3.5780 - val_accuracy: 0.1333
Epoch 5/10
6/6 [==============================] - 0s 48ms/step - loss: 2.6110 - accuracy: 0.1732 - val_loss: 3.3782 - val_accuracy: 0.1333
Epoch 6/10
6/6 [==============================] - 0s 47ms/step - loss: 2.5487 - accuracy: 0.1955 - val_loss: 3.2270 - val_accuracy: 0.1333
Epoch 7/10
6/6 [==============================] - 0s 48ms/step - loss: 2.4549 - accuracy: 0.2011 - val_loss: 3.0883 - val_accuracy: 0.1333
Epoch 8/10
6/6 [======================

6/6 [==============================] - 0s 18ms/step - loss: 2.5136 - accuracy: 0.1620 - val_loss: 3.9381 - val_accuracy: 0.0889
Epoch 7/10
6/6 [==============================] - 0s 16ms/step - loss: 2.4309 - accuracy: 0.1397 - val_loss: 3.7181 - val_accuracy: 0.0889
Epoch 8/10
6/6 [==============================] - 0s 15ms/step - loss: 2.4226 - accuracy: 0.1508 - val_loss: 3.5533 - val_accuracy: 0.0889
Epoch 9/10
6/6 [==============================] - 0s 17ms/step - loss: 2.3777 - accuracy: 0.1564 - val_loss: 3.4177 - val_accuracy: 0.0889
Epoch 10/10
6/6 [==============================] - 0s 15ms/step - loss: 2.3410 - accuracy: 0.1620 - val_loss: 3.3054 - val_accuracy: 0.0889
Processing data_60_13_4096_256.json
Training sets loaded!
input shape: 87 13
output shape: 10
Epoch 1/10
6/6 [==============================] - 0s 77ms/step - loss: 2.8060 - accuracy: 0.1173 - val_loss: 7.1438 - val_accuracy: 0.0667
Epoch 2/10
6/6 [==============================] - 0s 42ms/step - loss: 2.6145 - ac

6/6 [==============================] - 0s 48ms/step - loss: 3.1366 - accuracy: 0.0615 - val_loss: 4.0491 - val_accuracy: 0.1556
Epoch 2/10
6/6 [==============================] - 0s 21ms/step - loss: 2.9364 - accuracy: 0.0894 - val_loss: 3.5286 - val_accuracy: 0.1556
Epoch 3/10
6/6 [==============================] - 0s 22ms/step - loss: 2.7725 - accuracy: 0.0950 - val_loss: 3.2279 - val_accuracy: 0.1556
Epoch 4/10
6/6 [==============================] - 0s 20ms/step - loss: 2.6287 - accuracy: 0.1173 - val_loss: 3.0370 - val_accuracy: 0.1556
Epoch 5/10
6/6 [==============================] - 0s 21ms/step - loss: 2.4949 - accuracy: 0.1453 - val_loss: 2.8976 - val_accuracy: 0.1333
Epoch 6/10
6/6 [==============================] - 0s 22ms/step - loss: 2.4153 - accuracy: 0.1844 - val_loss: 2.7823 - val_accuracy: 0.1556
Epoch 7/10
6/6 [==============================] - 0s 24ms/step - loss: 2.3005 - accuracy: 0.2235 - val_loss: 2.6880 - val_accuracy: 0.1778
Epoch 8/10
6/6 [======================

6/6 [==============================] - 0s 36ms/step - loss: 2.5253 - accuracy: 0.1732 - val_loss: 3.2365 - val_accuracy: 0.0444
Epoch 7/10
6/6 [==============================] - 0s 34ms/step - loss: 2.4187 - accuracy: 0.2123 - val_loss: 3.0100 - val_accuracy: 0.0444
Epoch 8/10
6/6 [==============================] - 0s 35ms/step - loss: 2.3019 - accuracy: 0.1955 - val_loss: 2.8501 - val_accuracy: 0.0444
Epoch 9/10
6/6 [==============================] - 0s 34ms/step - loss: 2.2064 - accuracy: 0.2235 - val_loss: 2.7300 - val_accuracy: 0.0667
Epoch 10/10
6/6 [==============================] - 0s 34ms/step - loss: 2.1189 - accuracy: 0.2570 - val_loss: 2.6361 - val_accuracy: 0.0889
Processing data_60_20_4096_1024.json
Training sets loaded!
input shape: 22 20
output shape: 10
Epoch 1/10
6/6 [==============================] - 0s 45ms/step - loss: 3.1749 - accuracy: 0.0782 - val_loss: 4.1335 - val_accuracy: 0.0889
Epoch 2/10
6/6 [==============================] - 0s 19ms/step - loss: 3.0153 - a

6/6 [==============================] - 1s 85ms/step - loss: 3.3640 - accuracy: 0.1173 - val_loss: 7.0676 - val_accuracy: 0.2000
Epoch 2/10
6/6 [==============================] - 0s 54ms/step - loss: 3.0437 - accuracy: 0.1453 - val_loss: 5.3734 - val_accuracy: 0.1556
Epoch 3/10
6/6 [==============================] - 0s 53ms/step - loss: 2.7250 - accuracy: 0.1732 - val_loss: 4.4515 - val_accuracy: 0.1778
Epoch 4/10
6/6 [==============================] - 0s 53ms/step - loss: 2.5100 - accuracy: 0.2458 - val_loss: 3.8770 - val_accuracy: 0.1556
Epoch 5/10
6/6 [==============================] - 0s 54ms/step - loss: 2.3672 - accuracy: 0.2682 - val_loss: 3.5045 - val_accuracy: 0.1556
Epoch 6/10
6/6 [==============================] - 0s 54ms/step - loss: 2.2647 - accuracy: 0.2905 - val_loss: 3.2451 - val_accuracy: 0.1556
Epoch 7/10
6/6 [==============================] - 0s 54ms/step - loss: 2.1682 - accuracy: 0.2626 - val_loss: 3.0583 - val_accuracy: 0.1556
Epoch 8/10
6/6 [======================

Now that we have our estimates we will build a table sort them and print out the top 5 performers.

In [12]:
resultsdf = pd.DataFrame({}, columns=['File','Validation Accuracy'])
for row in results:
    data = {'File': row['filename'], 
            'Validation Accuracy': max(row['history'].history['val_accuracy'])}
    resultsdf = resultsdf.append(data, ignore_index=True)

resultsdf = resultsdf.sort_values(['Validation Accuracy'], ascending=False)
resultsdf.head()

,File,Validation Accuracy
21,data_0_30_2048_256.json,0.861045
25,data_0_30_4096_512.json,0.846793
24,data_0_30_4096_256.json,0.845606
15,data_0_20_4096_256.json,0.814727
18,data_0_30_1024_256.json,0.801663


Now that we know the best performer for the generic model will will extract the entire dataset using that performer.

In [13]:
json_file_name = 'full_data.json'

fname = resultsdf['File'].at[0]
opt = fname.split('_')
opt2 = opt[4].split('.')

a_length = int(opt[1])
mfccs = int(opt[2])
fft = int(opt[3])
h_length = int(opt2[0])

print('Audio Length:', a_length)
print('Num MFCCS:', mfccs)
print('N FFT:', fft)
print('Hop Length:', h_length)

print('building full dataset', json_file_name)
dsPath = DATASET_PATH

# Merge files to the specified length as needed
if a_length != 0:
    prep_wave_files(dsPath, WORK_PATH, a_length, max_subj=-1)
    dsPath = WORK_PATH

# Extract the features into the json file
preprocess_dataset(dsPath, json_file_name, num_mfcc=mfccs, n_fft=fft, hop_length=h_length, max_subj=-1)

Audio Length: 0
Num MFCCS: 13
N FFT: 1024
Hop Length: 256
building full dataset full_data.json
    Processing: 'p226'
    Processing: 'p302'
    Processing: 'p284'
    Processing: 'p239'
    Processing: 'p335'
    Processing: 'p263'
    Processing: 'p283'
    Processing: 'p282'
    Processing: 'p276'
    Processing: 'p362'
    Processing: 'p341'
    Processing: 'p323'
    Processing: 'p292'
    Processing: 'p238'
    Processing: 'p237'
    Processing: 'p234'
    Processing: 'p293'
    Processing: 'p271'
    Processing: 'p260'
    Processing: 'p255'
    Processing: 'p273'
    Processing: 'p265'
    Processing: 'p313'
    Processing: 'p232'
    Processing: 'p262'
    Processing: 'p314'
    Processing: 'p253'
    Processing: 'p316'
    Processing: 'p361'
    Processing: 'p254'
    Processing: 'p363'
    Processing: 'p288'
    Processing: 'p287'
    Processing: 'p305'
    Processing: 'p264'
    Processing: 'p307'
    Processing: 'p347'
    Processing: 'p241'
    Processing: 'p270'
    Proc